Longer term it would be preferable to download the more detailed match-by-match data for every team with shot counts, and develop a prediction model for the number of shots as well as the overall xG. This would allow a model to predict match results using simulations via Poisson distributions

For now, we are working with the less detailed summary data and only have xG total forecasts to work with. So, we will build a simple prediction model for win-draw-loss probabilities based on xG totals

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from autoscout.util import load_csv

In [2]:
epl_df = load_csv("../data/ratings/epl_2022.csv")

Build an outcome column in [0, 1, 2] denoting [home win, draw, away win]

In [3]:
epl_df["outcome"] = np.select(
    condlist=[
        epl_df["home_goals"] > epl_df["away_goals"],
        epl_df["home_goals"] < epl_df["away_goals"]
    ],
    choicelist=[0, 2],
    default=1,
)

In [4]:
epl_df.head()

,date,home_team,home_xg,away_xg,away_team,home_goals,away_goals,home_att_rating,home_def_rating,away_att_rating,away_def_rating,home_pred,away_pred,outcome
0,2021-08-13,Brentford,1.3,1.4,Arsenal,2,0,994.56,997.76,1002.24,1005.44,1.47,1.33,0
1,2021-08-14,Manchester Utd,1.5,0.6,Leeds United,5,1,1000.96,1023.36,976.64,999.04,1.47,1.33,0
2,2021-08-14,Watford,1.0,1.2,Aston Villa,3,2,984.96,1004.16,995.84,1015.04,1.47,1.33,0
3,2021-08-14,Chelsea,0.9,0.3,Crystal Palace,3,0,981.76,1032.96,967.04,1018.24,1.47,1.33,0
4,2021-08-14,Everton,2.4,0.7,Southampton,3,1,1029.76,1020.16,979.84,970.24,1.47,1.33,0


`X` and `y` data - we are not using a test set for now

In [5]:
X = epl_df[["home_pred", "away_pred"]]
y = epl_df["outcome"]

We define a cross validation strategy

In [6]:
def crossval(model, X_data, y_data) -> np.ndarray:
    cross_val = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, X_data, y_data, scoring="accuracy", cv=cross_val, n_jobs=-1)
    return n_scores

We try using a multiclass logistic regression model from SKLearn first

In [7]:
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=0.1)
n_scores = crossval(logistic_model, X, y)
print("Mean Accuracy: %.3f (%.3f)" % (np.mean(n_scores), np.std(n_scores)))

Mean Accuracy: 0.533 (0.062)


After cross validation shows reasonable results, we fit on the whole dataset

In [8]:
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=0.1).fit(X, y)

And evaluate...

In [9]:
print(accuracy_score(y, logistic_model.predict(X)))
print(np.unique(logistic_model.predict(X)))
print(logistic_model.predict_proba(X))

0.5342105263157895
[0 2]
[[0.43050573 0.24403915 0.32545512]
 [0.43050573 0.24403915 0.32545512]
 [0.43050573 0.24403915 0.32545512]
 ...
 [0.78095292 0.15436604 0.06468104]
 [0.7160753  0.18282859 0.10109611]
 [0.70102009 0.18710881 0.1118711 ]]


Not any draw (1) predictions, maybe we can make use of ordinal nature here

We define a simple class for building what is effectively an ordinal classifier from any SKLearn classification model

In [10]:
from sklearn.base import clone, BaseEstimator, ClassifierMixin

class OrdinalClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}

    def fit(self, X, y):
        self.uniques_class = np.sort(np.unique(y))
        if self.uniques_class.shape[0] > 2:
            for i in range(self.uniques_class.shape[0] - 1):
                binary_y = (y > self.uniques_class[1]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
        return self

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

    def predict_proba(self, X):
        clfs_predict = {
            k: self.clfs[k].predict_proba(X)
            for k in self.clfs
        }

        predicted = []
        for i, y in enumerate(self.uniques_class):
            if i == 0:
                predicted.append(1 - clfs_predict[y][:, 1])
            elif y in clfs_predict:
                predicted.append(clfs_predict[y - 1][:, 1] - clfs_predict[y][:, 1])
            else:
                predicted.append(clfs_predict[y - 1][:, 1])

        return np.vstack(predicted).T

We use `OrdinalClassifier` with logistic regression

In [11]:
ordinal_model = OrdinalClassifier(LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=0.1)).fit(X, y)

And evaluate...

In [12]:
print(accuracy_score(y, ordinal_model.predict(X)))
print(np.unique(ordinal_model.predict(X)))
print(ordinal_model.predict_proba(X))

0.5078947368421053
[0 2]
[[0.67945569 0.         0.32054431]
 [0.67945569 0.         0.32054431]
 [0.67945569 0.         0.32054431]
 ...
 [0.93650124 0.         0.06349876]
 [0.90242217 0.         0.09757783]
 [0.89169237 0.         0.10830763]]


Does not seem to help with the lack of draw predictions - in fact it seems to have made them worse. These predictions also seem to massively favour home teams

We will try oversampling draws using SMOTE

In [13]:
from imblearn.over_sampling import SMOTE

resampler = SMOTE(random_state=1)
X_res, y_res = resampler.fit_resample(X, y)

Now we go back to the logistic regression model using the resampled data

In [14]:
logistic_res_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=0.1).fit(X_res, y_res)

And evaluate, noting that we fit with the resampled data but evaluate on the original data...

In [15]:
print(accuracy_score(y, logistic_res_model.predict(X)))
print(np.unique(logistic_res_model.predict(X)))
print(logistic_res_model.predict_proba(X))

0.49736842105263157
[0 1 2]
[[0.33309589 0.35105453 0.31584958]
 [0.33309589 0.35105453 0.31584958]
 [0.33309589 0.35105453 0.31584958]
 ...
 [0.69336283 0.24267769 0.06395949]
 [0.61730144 0.28278115 0.0999174 ]
 [0.60284066 0.28638433 0.11077501]]


Accuracy is slightly down on the original model but the distribution looks a bit better initially

Next we try the ordinal model with the resampled data

In [16]:
ordinal_res_model = OrdinalClassifier(LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=0.1)).fit(X_res, y_res)

And evaluate on the original data...

In [17]:
print(accuracy_score(y, ordinal_res_model.predict(X)))
print(np.unique(ordinal_res_model.predict(X)))
print(ordinal_res_model.predict_proba(X))

0.5052631578947369
[0 2]
[[0.69009269 0.         0.30990731]
 [0.69009269 0.         0.30990731]
 [0.69009269 0.         0.30990731]
 ...
 [0.93733837 0.         0.06266163]
 [0.90445499 0.         0.09554501]
 [0.89362737 0.         0.10637263]]


The ordinal model does not seem to benefit, so the logistic regression seems preferable

However, oversampling seems to result in draw probabilities which are too high and reduce overall accuracy

For now, we will use the predictions from the original logistic regression but more research would be useful here

To finish we will convert the result forecast to xPts

In [18]:
epl_df[["home_prob", "draw_prob", "away_prob"]] = logistic_model.predict_proba(X)
epl_df["home_xpts"] = (3 * epl_df["home_prob"] + 1 * epl_df["draw_prob"])
epl_df["away_xpts"] = (3 * epl_df["away_prob"] + 1 * epl_df["draw_prob"])
epl_df.to_csv("../data/preds/epl_2022.csv", index=False)